In [1]:
# Author: Alexander Maksiaev
# Citation: https://medium.com/the-data-perspectives/workarounds-openai-models-token-limit-issues-3ea52a60d937
# Purpose: Have ChatGPT do sentiment analysis on the autism textbooks using the "map reduce" technique

In [2]:
# Housekeeping

import os
import docx
from openai import OpenAI
import pandas as pd
import re
# from langchain_openai import OpenAI
# from langchain.chains.question_answering import load_qa_chain
# # from langchain.document_loaders import PyPDFLoader
# from langchain.docstore.document import Document
# import docx2txt
# from langchain.chains.summarize import load_summarize_chain

key = 'YOUR KEY' # yourapi key here

client = OpenAI(api_key = key)


In [3]:
# Switch into textbook directory
os.getcwd()

textbook_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Textbook_Dump_Autism"

textbooks = os.listdir(textbook_dir)

# Go back to textbook directory 

os.chdir(textbook_dir)

# Function to get full text
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return fullText

# Dictionary filled with text for all the books, minus the titles
title_text = {}
for book in textbooks:
    total_text = getText(book)
    text_without_title = total_text[4:]
    for piece in text_without_title:
        if piece == '':
            text_without_title.remove(piece) # Does not get rid of all whitespace, but ah well.
    # Make it a list
#     text_without_title = list(text_without_title)
    title_text[book] = text_without_title

In [4]:
print(title_text["Abn_Barlow_07_Autism.docx"][0])

test_text = title_text["Abn_Barlow_07_Autism.docx"][0]

p. 524: Autism Spectrum Disorder Autism spectrum disorder (ASD) is a neurodevelopmental dis-order that, at its core, affects how one perceives and socializes with others (Durand, in press). DSM-5 combined most of the disorders previously included under the umbrella term "pervasive develop-mental disorders" (e.g., autistic disorder, Asperger's disorder, and childhood disintegrative disorder) and included them into this one category (American Psychiatric Association, 2013). Moreover, Rett disorder, a genetic condition that affects mostly females, is diagnosed as ASD with the qualifier "associated with Rett syndrome" or "associated with MeCP2 mutation" (the gene involved in Rett syndrome). And the designation "not otherwise specified" which was applied to other disorders prior to DSM-5, was deleted. A disorder new to DSM-5-Social (Pragmatic) Communication Disorder-includes the difficulties in social communication seen in ASD, but without restricted, repetitive patterns of behavior. These 

In [5]:
# chain = load_qa_chain(
#     llm, chain_type="map_reduce", verbose=True, return_intermediate_steps=True
# )
# query = "For every piece of text I send from now on, give a single sentiment 'score' between -4 and 4 for the following piece of text. -4 represents the most negative sentiment, while 4 represents the most positive sentiment. Give the score as a decimal with up to five significant figures."

# result = chain({"input_documents": text, "question": query}
# , return_only_outputs=True
# )
# print(result['output_text'])

In [6]:
# # Test to see if it works on one piece of the text



# query = "For every piece of text I send from now on, give a single sentiment 'score' between -4 and 4 for the following piece of text. -4 represents the most negative sentiment, while 4 represents the most positive sentiment. Give the score as a decimal with up to five significant figures."

# messages = [{"role": "user", "content": query}]
# messages.append({"role":"user", "content": test_text})

# answers = client.chat.completions.create(
#         model = "gpt-3.5-turbo",
#         messages = messages
# )

# print(answers.choices[0].message.content)


# # Test
# # answers = client.chat.completions.create(
# #     messages=[{
# #         "role": "user",
# #         "content": "Say this is a test",
# #     }],
# #     model="gpt-3.5-turbo",
# # )

In [7]:
# # Test to see if it works on the entire textbook
# to_append_to_messages = []
# for i in range(len(title_text["Abn_Barlow_07_Autism.docx"])):
#     to_append_to_messages.append(title_text["Abn_Barlow_05_Autism_v2.docx"][i])
    
test_query = "For every piece of text I send from now on, give a single sentiment 'score' between -4 and 4 for the following piece of text. -4 represents the most negative sentiment, while 4 represents the most positive sentiment. Give the score as a decimal with up to five significant figures. Do not give any other text except for the score."

def one_textbook(textbook_appendage, query):
    answers = []
    for to_append in textbook_appendage:
        messages = [{"role": "user", "content": query}]
        messages.append({"role":"user", "content": to_append})

        answer = client.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages = messages
        )

        answers.append(answer.choices[0].message.content)
        
    total_textbook_sentiment = 0
    extra = 0 # If there are extra sentiments hiding, make sure the average isn't off
    for sentiment in answers:
        sentiment = re.findall(r"[-+]?(?:\d*\.*\d+)", sentiment)
        if len(sentiment) > 0: # If gpt gave a score
            for s in sentiment:
                s = float(s)
                total_textbook_sentiment += s # Add them all up
        if len(sentiment) > 1:
            extra += len(sentiment) - 1
    total_textbook_sentiment = total_textbook_sentiment/(len(answers) + extra) # Average the sentiments
        
    return total_textbook_sentiment

# test_answers = one_textbook(to_append_to_messages, test_query)

# print(test_answers)


    
# print(total_textbook_sentiment)

In [8]:
# Another test

to_append_to_messages_test = []
for i in range(len(title_text["Abn_Barlow_05_Autism_v2.docx"])):
    to_append_to_messages_test.append(title_text["Abn_Barlow_05_Autism_v2.docx"][i])
    
print(one_textbook(to_append_to_messages_test, test_query))

-1.2690753086419755


In [9]:
# Function to do all of the textbooks

def all_textbooks(textbook_dict, query):
    sentiments = {}
    for textbook in textbook_dict:
        print("textbook: ", textbook)
        to_append_to_messages = []
        for i in range(len(textbook_dict[textbook])):
#             print("thing: ", i)
            to_append_to_messages.append(textbook_dict[textbook][i])
        sentiment = one_textbook(to_append_to_messages, query)
        sentiments[textbook] = sentiment
    return sentiments

In [10]:
all_sentiments = all_textbooks(title_text, test_query)

textbook:  Abn_Barlow_04_Autism_v2.docx
textbook:  Abn_Barlow_05_Autism_v2.docx
textbook:  Abn_Barlow_06_Autism_v2.docx
textbook:  Abn_Barlow_07_Autism.docx
textbook:  Abn_Barlow_08_Autism.docx
textbook:  Abn_Brown_01_Autism_v2.docx
textbook:  Abn_Brown_02_Autism_v2.docx
textbook:  Abn_Brown_03_Autism_v2.docx
textbook:  Abn_Brown_04_Autism_v2.docx
textbook:  Abn_Brown_05_Autism.docx
textbook:  Abn_Comer_05_Autism_v2.docx
textbook:  Abn_Comer_06_Autism_v2.docx
textbook:  Abn_Comer_07_Autism_v2.docx
textbook:  Abn_Comer_08_Autism_v2.docx
textbook:  Abn_Comer_09_Autism.docx
textbook:  Abn_Hooley_13_Autism_v2.docx
textbook:  Abn_Hooley_14_Autism_v2.docx
textbook:  Abn_Hooley_15_Autism_v2.docx
textbook:  Abn_Hooley_16_Autism_v2.docx
textbook:  Abn_Hooley_17_Autism.docx
textbook:  Abn_Kearney_06_Autism.docx
textbook:  Abn_Mash_02_Autism_v2.docx
textbook:  Abn_Mash_03_Autism_v2.docx
textbook:  Abn_Mash_04_Autism_v2.docx
textbook:  Abn_Mash_05_Autism_v2.docx
textbook:  Abn_Mash_06_Autism.docx


textbook:  Spcl_Turnbull_05_Autism_v2.docx
textbook:  Spcl_Turnbull_06_Autism_v2.docx
textbook:  Spcl_Turnbull_07_Autism_v2.docx
textbook:  Spcl_Turnbull_08_Autism.docx
textbook:  Spcl_Vaughn_03_Autism_v2.docx
textbook:  Spcl_Vaughn_04_Autism_v2.docx
textbook:  Spcl_Vaughn_05_Autism_v2.docx
textbook:  Spcl_Vaughn_06_Autism_v2.docx
textbook:  Spcl_Vaughn_07_Autism.docx


In [11]:
dmp_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\GitHub_DMP\Results"

os.chdir(dmp_dir)

all_sentiments_df = pd.DataFrame(all_sentiments, index = [0])
all_sentiments_df.to_csv('chatgpt_sentiments_autism_03_09_2024.csv', encoding='utf-8')